# Deep Face Implementation

In [1]:
# imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import cv2
from ast import literal_eval

import keras
import tensorflow as tf

In [2]:
from deepface import DeepFace

In [3]:
# file paths
path = os.getcwd()
path = path[:-4]
data_dir = path + "/data/lfw-deepfunneled"

In [4]:
matchPairsTrain, mismatchPairsTrain = pd.read_csv(path + "/data/matchpairsDevTrain.csv"), pd.read_csv(path + "/data/mismatchpairsDevTrain.csv")
matchPairsTest, mismatchPairsTest = pd.read_csv(path + "/data/matchpairsDevTest.csv"), pd.read_csv(path + "/data/mismatchpairsDevTest.csv")

In [5]:
def getImg(name, num):
    imgPath = data_dir + '/' + name + '/' + name + '_' +  "{:04d}".format(num) + '.jpg'
    img = cv2.imread(imgPath)
    img = cv2.resize(img, (299, 299))
    return img

def matchPairs(X):
    df = X.copy(deep=False)
    imageCol1, imageCol2 = [], []
    for _ , person in df.iterrows():
        name = person["name"]
        img1, img2 = person.imagenum1, person.imagenum2
        img1, img2 = getImg(name, img1), getImg(name, img2)
        imageCol1.append(img1)
        imageCol2.append(img2)
    df["image1"] = imageCol1
    df["image2"] = imageCol2
    return df

def mismatchPairs(X):
    df = X.copy(deep=False)
    imageCol1, imageCol2 = [], []
    for _ , person in df.iterrows():
        name, name1 = person["name"], person["name.1"]
        img1, img2 = person.imagenum1, person.imagenum2
        img1, img2 = getImg(name, img1), getImg(name1, img2)
        imageCol1.append(img1)
        imageCol2.append(img2)
    df["image1"] = imageCol1
    df["image2"] = imageCol2
    return df

In [6]:
matchPairsTrain = matchPairs(matchPairsTrain)
matchPairsTest = matchPairs(matchPairsTest)
mismatchPairsTrain = mismatchPairs(mismatchPairsTrain)
mismatchPairsTest = mismatchPairs(mismatchPairsTest)

In [7]:
matchPairsTest.tail()

,name,imagenum1,imagenum2,image1,image2
495,Winona_Ryder,6,15,"[[[8, 3, 0], [8, 3, 0], [7, 2, 0], [7, 2, 0], ...","[[[0, 0, 3], [0, 0, 3], [0, 0, 3], [0, 0, 3], ..."
496,Winona_Ryder,19,21,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 6], [0, 0, 4], [0, 0, 3], [0, 0, 2], ..."
497,Yevgeny_Kafelnikov,3,4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
498,Yoriko_Kawaguchi,3,10,"[[[247, 221, 205], [247, 221, 205], [247, 221,...","[[[0, 0, 4], [0, 0, 3], [0, 0, 1], [1, 0, 0], ..."
499,Zoran_Djindjic,3,4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 0, 1], ..."


In [8]:
mismatchPairsTest.tail()

,name,imagenum1,name.1,imagenum2,image1,image2
495,Susan_Whelan,1,Wolfgang_Schneiderhan,1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
496,Takeo_Fukui,1,Will_Ofenheusle,1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[2, 0, 0], [2, 0, 0], [2, 0, 0], [2, 0, 0], ..."
497,Tamara_Mowry,1,Zach_Parise,1,"[[[0, 2, 0], [146, 157, 155], [196, 210, 208],...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
498,Tatiana_Kennedy_Schlossberg,1,Thomas_Watjen,1,"[[[0, 2, 0], [0, 1, 0], [1, 0, 0], [3, 0, 0], ...","[[[5, 0, 0], [5, 0, 0], [5, 0, 0], [5, 0, 0], ..."
499,Todd_Petit,1,Vicente_Fernandez,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


Models: Facenet, Facenet512, OpenFace, DeepID, SFace, GhostFaceNet

In [9]:
models = ["Facenet", "Facenet512", "OpenFace", "DeepID", "SFace", "GhostFaceNet"]

In [10]:
def run_confusion_matrix(model):
    matches = []
    mismatches = []
    
    for _ , person in matchPairsTest.iterrows():
        try:
            result = DeepFace.verify(person.image1, person.image2, model_name=model)
        except ValueError:
            continue
        matches.append(result['verified'])

    for _ , person in mismatchPairsTest.iterrows():
        try:
            result = DeepFace.verify(person.image1, person.image2, model_name=model)
        except ValueError:
            continue
        mismatches.append(result['verified'])
        
    matches = pd.DataFrame(matches, columns=["Result"])
    mismatches = pd.DataFrame(mismatches, columns=["Result"])

    tp = len(matches[matches.Result == True])
    fn = len(matches[matches.Result == False])
    fp = len(mismatches[mismatches.Result == True])
    tn = len(mismatches[mismatches.Result == False])
    
    return [[tp, fn], [fp, tn]]

In [12]:
results = []
for model in models:
    results.append([model, run_confusion_matrix(model)])
results = pd.DataFrame(results, columns=["Model", "Confusion Matrix"])

In [13]:
results

,Model,Confusion Matrix
0,Facenet,"[[321, 163], [5, 468]]"
1,Facenet512,"[[247, 237], [0, 473]]"
2,OpenFace,"[[13, 471], [1, 472]]"
3,DeepID,"[[40, 444], [8, 465]]"
4,SFace,"[[351, 133], [2, 471]]"
5,GhostFaceNet,"[[340, 144], [3, 470]]"


In [15]:
results.to_csv(path + "/results/DeepFace_results.csv")